<a href="https://colab.research.google.com/github/kravtsova2/kravtsova2.github.io/blob/master/ECG_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ################################# convert ECG to images and back to arrays of needed shape
############################### this is the end of trying on images. Now try on our data
### load 2D CNN model
import tensorflow as tf
import cv2

import pandas as pd
import numpy as np
from numpy.linalg import norm

import seaborn as sns ### to plot heatmap
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model
from scipy import stats

from google.colab import files
uploaded = files.upload()

Saving x_test.npy to x_test (1).npy
Saving x_train.npy to x_train (1).npy
Saving y_test.npy to y_test.npy
Saving y_train.npy to y_train.npy


In [5]:
# Load the array from the files
filename_xtrain = "x_train.npy"
filename_xtest = "x_test.npy"
filename_ytrain= "y_train.npy"
filename_ytest= "y_test.npy"

x_train = np.load(filename_xtrain)
x_test = np.load(filename_xtest)
y_train = np.load(filename_ytrain)
y_test = np.load(filename_ytest)


In [6]:
NN=200 #size of reshaped image
def to_img_and_back(Xdat):
    fig, axes = plt.subplots(nrows=12, ncols=1, figsize=(4,4), sharex=True)
    lead_names = ["I", "II", "III", "aVR", "aVL", "aVF", "V1", "V2", "V3", "V4", "V5", "V6"]
    for i, ax in enumerate(axes):
           ax.plot(stats.zscore(Xdat[i,:]),color='black')
           ax.set_yticks([]) # Hide y-axis ticks
           plt.tight_layout()
           ax.set_xticks([]) # Hide y-axis ticks

     # Save the plot in black and white, low resolution (e.g., 100 dpi)
    plt.savefig('plot_bw_lowres.png', dpi=100, format='png', facecolor='white', edgecolor='none')
    img_array=cv2.imread("plot_bw_lowres.png") # read image
    gray_img=cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    plt.close(fig)
    return gray_img


x_train_new=[]
for i in range(x_train.shape[0]):
    Xdat=x_train[i,:,:]
    Xdat_new=to_img_and_back(Xdat)
    x_train_new.append(Xdat_new)
x_train_new=np.array(x_train_new)

### rename
x_train=x_train_new



In [7]:
### reshape x, make use y is categorical
x_train = tf.expand_dims(x_train, axis=-1)
x_train = tf.divide(x_train, 255)

# y set
y_train = (y_train % 2 == 0).astype(int)

print(x_train.shape, y_train.shape)

NN=x_train.shape[1]


(200, 400, 400, 1) (200,)


In [9]:
############################### define the model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dropout,
                                     Dense, GlobalAveragePooling2D)

model = Sequential()

#Adding first convolutional layer
model.add(Conv2D(16, kernel_size=(3,3), input_shape = (NN,NN,1)))
model.add(Conv2D(32, kernel_size=(3,3), activation="relu"))
model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(Conv2D(128, kernel_size=(3,3), activation="relu",name='my_last_conv'))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(2, activation=tf.nn.softmax,name='my_dense'))
model.summary()

model.compile(optimizer = "adam",
              loss = "sparse_categorical_crossentropy",
              metrics = ["accuracy"])


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 398, 398, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 396, 396, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 394, 394, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ my_last_conv (Conv2D)           │ (None, 392, 392, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ my_dense (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,410 (380.51 KB)

 Trainable params: 97,410 (380.51 KB)

 Non-trainable params: 0 (0.00 B)